# Inventory or Workforce Optimization
Choose one scenario and run sensitivity ±10 percent.

In [ ]:

from pathlib import Path
import pandas as pd
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, LpStatus, lpSum, LpInteger, LpContinuous, LpBinary, PULP_CBC_CMD

# Example: Workforce planning over 7 days
days = list(range(7))
demand = {d:int(50 + 10*np.sin(d)) for d in days}  # required worker-hours per day
max_hours_per_worker = 8
overtime_cost = 1.5
base_cost = 1.0

prob = LpProblem("WorkforcePlanning", LpMinimize)

# Decision variables: number of workers scheduled each day (integer), overtime hours (continuous)
workers = {d: LpVariable(f"workers_{d}", lowBound=0, cat=LpInteger) for d in days}
ot_hours = {d: LpVariable(f"ot_{d}", lowBound=0, cat=LpContinuous) for d in days}

# Objective: minimize cost
prob += lpSum([base_cost * workers[d] * max_hours_per_worker + overtime_cost * ot_hours[d] for d in days])

# Constraints: cover demand
for d in days:
    prob += workers[d]*max_hours_per_worker + ot_hours[d] >= demand[d]

# Solve
prob.solve(PULP_CBC_CMD(msg=False))
status = LpStatus[prob.status]
print("Status:", status)

sol = pd.DataFrame({
    "day": days,
    "workers": [workers[d].value() for d in days],
    "overtime_hours": [ot_hours[d].value() for d in days],
    "demand": [demand[d] for d in days]
})
sol["base_capacity"] = sol["workers"] * max_hours_per_worker
sol["total_capacity"] = sol["base_capacity"] + sol["overtime_hours"]
print(sol)

# Sensitivity: vary demand ±10 percent and record objective
def run_with_scale(scale):
    demand2 = {d:int(demand[d]*scale) for d in days}
    prob2 = LpProblem("WF_Sensitivity", LpMinimize)
    workers2 = {d: LpVariable(f"w_{d}", lowBound=0, cat=LpInteger) for d in days}
    ot2 = {d: LpVariable(f"ot_{d}", lowBound=0, cat=LpContinuous) for d in days}
    prob2 += lpSum([base_cost * workers2[d]*max_hours_per_worker + overtime_cost*ot2[d] for d in days])
    for d in days:
        prob2 += workers2[d]*max_hours_per_worker + ot2[d] >= demand2[d]
    prob2.solve(PULP_CBC_CMD(msg=False))
    obj = prob2.objective.value()
    return obj

for scale in [0.9, 1.0, 1.1]:
    obj = run_with_scale(scale)
    print(f"Objective at demand scale {scale}: {obj:.2f}")
